In [48]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from hyperion.model import ModelOutput
import astropy.constants as const

AU = const.au.cgs.value
pc = const.pc.cgs.value
MS = const.M_sun.cgs.value
mh = const.m_p.cgs.value + const.m_e.cgs.value
dstar = 178. * pc

In [17]:
filename = '/Users/yaolun/bhr71/hyperion/model12.rtout'
m = ModelOutput(filename)

q = m.get_quantities()
r_wall, t_wall, p_wall = q.r_wall, q.t_wall, q.p_wall
rc     = 0.5*( r_wall[0:len(r_wall)-1] + r_wall[1:len(r_wall)] )
thetac = 0.5*( t_wall[0:len(t_wall)-1] + t_wall[1:len(t_wall)] )
phic   = 0.5*( p_wall[0:len(p_wall)-1] + p_wall[1:len(p_wall)])

rho = q['density'][0].array.T

In [3]:
def cell_size(gridi, q):
    # grid index in cell coordinates
    ir, itheta, iphi = gridi
    
    ri, thetai, phii = q.r_wall, q.t_wall, q.p_wall
    
    vol = (1/3.)*(ri[ir+1]**3 - ri[ir]**3) * (phii[iphi+1]-phii[iphi]) * -(np.cos(thetai[itheta+1])-np.cos(thetai[itheta]))
    
    return vol

## Diffraction limited beam

### 1.2 $\mu$m - 0.124 arcsec

### 1.6 $\mu$m - 0.165 arcsec

In [63]:
# beam size
beam = 0.124
beam_rad = beam/2 * dstar/pc * AU # beam radius in AU
rad_limit = beam/2*dstar/pc*AU/np.sin(2*np.pi/len(thetac))
mass_column = 0

# Projected distance on yz-plane -> edge-on view
dist_yz = np.empty_like(rho) 
for ir in range(len(rc)):
# for ir in range(100):
    # avoid empty cell
    if rc[ir] <= 0.1*AU:
        continue
    for it in range(len(thetac)):
        for ip in range(len(phic)):
            dist_yz[ir,it,ip] = ((rc[ir]*np.sin(thetac[it])*np.sin(phic[ip]))**2 * (rc[ir]*np.cos(thetac[it]))**2)**0.5
            
            if dist_yz[ir,it,ip] <= beam_rad:
                # If the projected spacing between adjacent cell in polar angle is smaller than the beam radius,
                # the cell is considered "entirely" within the beam.  Its density is calculated and added to column
                # density.  If the projected spacing between adjacent cell in polar angle is greater than the beam
                # radius, the cel is considered "more extended" than the beam.  The added density is calculated by
                # the density of the cell times the segment of column that falls in the cell.
                if rc[ir] <= rad_limit:
                    mass_column += rho[ir,it,ip] * cell_size((ir,it,ip),q)
    if rc[ir] > rad_limit:
        mass_column += (r_wall[ir+1]-r_wall[ir]) * np.pi*rad_limit**2 * rho[ir,199,0]

In [64]:
print mass_column  # this is dust
gas_column = mass_column*100.
Nh = gas_column/mh / (np.pi*rad_limit**2)
print mass_column/(np.pi*rad_limit**2), Nh

1.69390725785e+30
0.00488055917624 2.91632134326e+23


In [14]:
print np.shape(rho)
print np.shape(r_wall), np.shape(t_wall), np.shape(p_wall)
f = np.sin(2*np.pi/400.)/AU / 178.
print 0.124/f/AU

(710, 400, 50)
(711,) (401,) (51,)
1405.20494759
